In [1]:
import json
import pandas as pd
import sqlalchemy as sql

dbParams = json.load(open('../database-research_ro.json'))
connectString = f'mysql+pymysql://{dbParams["USER"]}:{dbParams["PASSWORD"]}@{dbParams["HOST"]}:{dbParams["PORT"]}/{dbParams["NAME"]}'
engine = sql.create_engine(connectString)

In [2]:
numberOfMonths = 4

courseQuery = f'''
SELECT
  DISTINCT ca.course_id as id,
  cc.name as name
FROM
  mwrite_peer_review.canvas_assignments ca
LEFT JOIN mwrite_peer_review.canvas_courses cc ON
  cc.id = ca.course_id
WHERE
  is_peer_review_assignment = 1
  AND ca.due_date_utc BETWEEN NOW() - INTERVAL {numberOfMonths} MONTH AND NOW()
ORDER BY
  ca.due_date_utc DESC
'''


def courseFilter(courseDF):
    courseIDList = []

    for index,row in courseDF.iterrows():
        if '22' in row['name']:
            courseIDList.append(row['id'])

    return courseIDList
    
def queryMaker(courseIDList, templateQuery = 'testQuery.sql'):
    with open(templateQuery) as queryFile:
        queryLines = ''.join(queryFile.readlines())

    # startIndex = 0
    # for index, line in enumerate(queryLines):
    #     if line.startswith('SELECT'):
    #         startIndex = index

    courseIDString = ',\n   '.join(map(str, courseIDList))+'\n'

    newQuery = queryLines.format(courseIDString)

    return newQuery

# def queryAsDictMaker(courseIDList, templateQuery = 'testQuery.sql'):
#     return {str(courseID):queryMaker([courseID], templateQuery) for courseID in courseIDList}


# def pullAndSaveQuery(saveName, query, engine, savePath='../TSVs/'):
#     with engine.connect() as connection:   
#         reviewDF = pd.read_sql(query, connection)
    
#     #print(saveName)
#     #display(reviewDF)
#     reviewDF.to_csv(savePath+saveName+'.tsv', sep="\t", quoting=3, quotechar="", escapechar="\\")
    
# def queryDictPullandQuery(queryDict, engine):
#     for id,query in queryDict.items():
#         pullAndSaveQuery(id,query,engine)

def queryBulkPull(query, engine):
    with engine.connect() as connection:   
        reviewDF = pd.read_sql(query, connection)

    return reviewDF


def saveQueries(courseDF, reviewDF, savePath='../TSVs/'):
    for _, row in courseDF.iterrows():
        (courseID, courseName) = row
        outputFilename = f'{courseID} - {courseName}.tsv'

        saveDF = reviewDF[reviewDF['CourseID'] == courseID]
        saveDF.to_csv(savePath+outputFilename, sep="\t", quoting=3, quotechar="", escapechar="\\")

In [3]:
with engine.connect() as connection:
    courseDF = pd.read_sql(courseQuery, connection)

newQuery = queryMaker(courseDF['id'])

reviewDF = queryBulkPull(newQuery, engine)

saveQueries(courseDF, reviewDF)